### category test

In [3]:
import sys
sys.path.append('/Users/jiechen/Apoex/Email/fastAPI')
from app.services.utils import fetchFromDB
from app.services.base_service import BaseService
from app.dataset.email_dataset import EmailDataset
import pandas as pd

bs = BaseService()
id = 202861
email = fetchFromDB(bs.email_spec_query.format(COND=f"e.id={id}"))
# email = pd.read_json(f'data/test_data/email8.json')
# email.to_json(f'data/test_data/cate{id}.json', orient="records", indent=4, force_ascii=False)
ed = EmailDataset(df=email)
ca = ed.do_connect()
ca

,id,from,sender,source,to,receiver,category,errandId,totalAmount,settlementAmount,reference,insuranceCaseRef,insuranceNumber,damageNumber,animalName,ownerName,note,connectedCol,showPage,isStaffAnimal
0,202861,kontakt@ahlbergsvet.se,Ahlbergs Veterinärpraktik,Clinic,mail@direktregleringsportalen.se,DRP,Question,[],<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Unreliable,None,Koppla,False


In [4]:
from app.services.classifier import Classifier
a = Classifier()
a.statistic(ca)

all_records: 858, auto_categorized: 810, 94.41%, manual: 48, 5.59%, 
                  connected_to_errands: 710, 82.75%, single_errandId: 710, 82.75%,                   reliable_connection: 622, 72.49%, staff_animal: 0, 0.00%
 - Auto_Reply: 71, connect:47, un-connect: 24
 - Finance_Report: 6, connect:0, un-connect: 6
 - Wisentic_Error: 15, connect:15, un-connect: 0
 - Other: 2, connect:0, un-connect: 2
 - Information: 0, connect:0, un-connect: 0
 - Insurance_Validation_Error: 5, connect:4, un-connect: 1
 - Settlement_Denied: 4, connect:4, un-connect: 0
 - Settlement_Approved: 506, connect:498, un-connect: 8
 - Settlement_Request: 7, connect:0, un-connect: 7
 - Message: 7, connect:6, un-connect: 1
 - Question: 8, connect:5, un-connect: 3
 - Spam: 1, connect:0, un-connect: 1
 - Complement_DR_Insurance_Company: 49, connect:49, un-connect: 0
 - Complement_Damage_Request_Insurance_Company: 56, connect:2, un-connect: 54
 - Complement_DR_Clinic: 57, connect:57, un-connect: 0
 - Complement_Da

{'total_records': 858,
 'auto_categorized': '810 (94.4%)',
 'manual_handling_required': '48 (5.6%)',
 'connected_to_errands': '710 (82.8%)',
 'single_errand_connection': '710 (82.8%)',
 'reliable_connections': '622 (72.5%)',
 'staff_animals': '0 (0.0%)'}

### forwarding test

In [1]:
import pandas as pd
from app.services.services import DefaultServices
from app.dataset.forward_dataset import ForwardDataset
id = 202126
userId = 14
df = pd.DataFrame([{ 'id': id, 'userId': userId}])
ds = ForwardDataset(df=df, services=DefaultServices())
forward = ds.do_forward()
forward

,id,action,forward_address,forward_subject,forward_text,journal_data
0,202126,Journalkopia,info@petvet.se,Info från försäkringsbolaget: Direktreglering ...,"Hej,<br><br>Här kommer önskemål om kompletteri...","{'insuranceCompany': 15, 'clinic': 87, 'journa..."


In [2]:
print(forward['forward_text'].iloc[0])

Hej,<br><br>Här kommer en fråga om en direktreglering från avsändaren. Svara på detta mail eller skicka ett mail med kompletteringen till 1000763357 så vidarebefordrar vi det till försäkringsbolaget.<br><br><b><i>"Hej!<br><br> Länken har utgått. Kan ni skicka en ny?<br><br> Tusen tack!<br><br> Mvh<br><br> Hanna<br><br> Skickat från min iPhone"</i></b><br><br>Tack på förhand!<br><br><b>Ärendesammanfattning:</b><br>Referens: 1000763357<br>Fakturanummer: 11250004726<br><br>Mvh Jie<br>DRP


### payment matching

In [ ]:
from app.services.utils import fetchFromDB
from app.dataset.payment_dataset import PaymentDataset
import pandas as pd

id = 64271
pay = fetchFromDB((bs.queries['payment'].iloc[0]).format(ID=id))
# pay = pd.read_json("data/test_data/pay.json")
ds = PaymentDataset(df=pay)
a = ds.do_match()
# ps.statistics(df)
print(a['status'].iloc[0])

One DR matched perfectly (reference: <a href="https://admin.direktregleringsportalen.se/errands/39449751047" target="_blank" style="background-color: gray; color: white; padding: 2px 5px;" title="matched by 1000747764">1000747764</a>).


### multiple visit

In [21]:
er = fetchFromDB((bs.queries['errandConnect'].iloc[0]).format(CONDITION="er.\"createdAt\" BETWEEN '2025-01-01' AND '2025-12-31' AND c.\"name\" ILIKE '%Hälsinge Smådjursklinik Hudiksvall%'"))
print(er.columns)
groups = er.groupby(['ownerName', 'animalName', 'insuranceCompany'])
result = []
i = 1
for key, group in groups:
    group_sorted = group.sort_values('date')
    group_sorted['diff_days'] = group_sorted['date'].diff().dt.days
    filtered = group_sorted[group_sorted['diff_days'] <= 5]
    if not filtered.empty and (i == 1):
        print("group_sorted:\n", group_sorted, "\nfiltered: \n", filtered)
        i += 1

    if not filtered.empty:
        result.append(filtered)

result_df = pd.concat(result) if result else pd.DataFrame()

print(result_df)


Index(['errandId', 'errandNumber', 'date', 'insuranceCompany', 'clinicName',
       'totalAmount', 'settlementAmount', 'reference', 'insuranceNumber',
       'damageNumber', 'invoiceReference', 'animalName', 'ownerName',
       'paymentOption', 'strategyType', 'settled'],
      dtype='object')
group_sorted:
      errandId errandNumber                             date insuranceCompany  \
608     68709  16220687095 2025-06-09 13:54:56.335324+00:00               If   
587     68983  16220689836 2025-06-12 08:07:48.508955+00:00               If   
115     74017  16220740175 2025-08-19 12:56:19.137794+00:00               If   

                             clinicName  totalAmount  settlementAmount  \
608  Hälsinge Smådjursklinik Hudiksvall       1592.0            1353.0   
587  Hälsinge Smådjursklinik Hudiksvall       4374.0            3718.0   
115  Hälsinge Smådjursklinik Hudiksvall       2226.0            1748.0   

      reference   insuranceNumber damageNumber invoiceReference animalNa

In [98]:
import pandas as pd
from datetime import timedelta
query = (f"er.\"createdAt\" BETWEEN '2025-01-01' AND '2025-12-31'" 
         f"AND c.\"name\" ILIKE '%Hälsinge Smådjursklinik Hudiksvall%'"
         f"AND es.cancelled = FALSE")
er = fetchFromDB((bs.queries['errandConnect'].iloc[0]).format(CONDITION=query))
WINDOW = pd.Timedelta(days=5)

def fmt_delta(td: pd.Timedelta) -> str:
    total_seconds = int(td.total_seconds())
    if total_seconds < 0:
        total_seconds = -total_seconds
    days, rem = divmod(total_seconds, 24*3600)
    hours, rem = divmod(rem, 3600)
    minutes, _ = divmod(rem, 60)
    parts = []
    if days:   parts.append(f"{days}day")
    if hours:  parts.append(f"{hours}hour")
    if minutes:parts.append(f"{minutes}min")
    if not parts:  # 小于1分钟
        parts = ["0min"]
    return "".join(parts)

er['createdAt'] = pd.to_datetime(er['date'], errors='coerce', utc=True).dt.tz_convert('Europe/Stockholm')
filtered_list = []

for key, grp in er.groupby(['ownerName', 'animalName', 'insuranceCompany'], dropna=False):
    g = grp.sort_values('createdAt').copy()
    g['delta_to_prev'] = g['createdAt'].diff()
    mask = g['delta_to_prev'] <= WINDOW
    if not mask.any():
        continue

    g['gap_label'] = ""
    g.loc[mask, 'gap_label'] = g.loc[mask, 'delta_to_prev'].apply(fmt_delta)
    prev_idx = g.index[mask] - 1  
    prev_index = g.loc[mask].index.map(lambda i: g.index[g.index.get_loc(i)-1])

    for i in prev_index:
        if g.at[i, 'gap_label'] == "":
            g.at[i, 'gap_label'] = "first"

    filtered_list.append(g[g['gap_label'] != ""])

filtered = pd.concat(filtered_list, ignore_index=False) if filtered_list else pd.DataFrame()
filtered = filtered.sort_values(['ownerName','animalName','insuranceCompany','createdAt'])
filtered.head()

,errandId,errandNumber,date,insuranceCompany,clinicName,totalAmount,settlementAmount,reference,insuranceNumber,damageNumber,invoiceReference,animalName,ownerName,paymentOption,strategyType,settled,createdAt,delta_to_prev,gap_label
612,68709,16220687095,2025-06-09 13:54:56.335324+00:00,If,Hälsinge Smådjursklinik Hudiksvall,1592.0,1353.0,1000684066,KD223469-8393-01,255282131,11250006374,Jenna,Agneta Nyström,fortus,creditcheck,True,2025-06-09 15:54:56.335324+02:00,NaT,first
592,68983,16220689836,2025-06-12 08:07:48.508955+00:00,If,Hälsinge Smådjursklinik Hudiksvall,4374.0,3718.0,1000686756,KD223469-8393-01,255282131,11250006534,Jenna,Agneta Nyström,fortus,creditcheck,True,2025-06-12 10:07:48.508955+02:00,2 days 18:12:52.173631,2day18hour12min
57,75065,39433750658,2025-09-01 12:46:07.492328+00:00,Agria,Hälsinge Smådjursklinik Hudiksvall,2620.0,0.0,1000747384,1236985-003-001,7575739-1,11250009989,Mini,Anders Eriksson,fortus,creditcheck,True,2025-09-01 14:46:07.492328+02:00,NaT,first
54,75098,39433750989,2025-09-02 07:19:39.908932+00:00,Agria,Hälsinge Smådjursklinik Hudiksvall,699.0,594.0,1000747707,1236985-003-001,7576557-1,11250010009,Mini,Anders Eriksson,swedbankPay,settlement,True,2025-09-02 09:19:39.908932+02:00,0 days 18:33:32.416604,18hour33min
213,73031,38568730311,2025-08-06 07:56:10.429513+00:00,Folksam,Hälsinge Smådjursklinik Hudiksvall,11727.0,5976.0,1000727139,CV-2083026-993,FF093566214S,11250008912,Hilja,Anders Svahn,manual,settlement,True,2025-08-06 09:56:10.429513+02:00,NaT,first


In [99]:
e1 = filtered.loc[filtered['gap_label']=='first', ['errandNumber', 'date', 'insuranceNumber', 'damageNumber', 'invoiceReference','animalName','ownerName','insuranceCompany']]
e2 = filtered.loc[filtered['gap_label']!='first', ['errandNumber', 'date', 'insuranceNumber', 'damageNumber', 'invoiceReference','animalName','ownerName','insuranceCompany']]
e = e1.merge(e2, on=['animalName','ownerName','insuranceCompany'], how = 'left', suffixes=['_before','_after'])
e.loc[:,['insuranceNumber_consistant','damageNumber_consistant','invoiceReference_consistant']] = False

for idx, row in e.iterrows():
    if row['insuranceNumber_before'] == row['insuranceNumber_after']:
        e.at[idx, 'insuranceNumber_consistant'] = True  
    if row['damageNumber_before'] == row['damageNumber_after']:
        e.at[idx, 'damageNumber_consistant'] = True
    if row['invoiceReference_before'] == row['invoiceReference_after']:
        e.at[idx, 'invoiceReference_consistant'] = True
e[['animalName','ownerName','insuranceCompany','errandNumber_before','date_before','errandNumber_after','date_after','insuranceNumber_consistant','damageNumber_consistant','invoiceReference_consistant']].head()

,animalName,ownerName,insuranceCompany,errandNumber_before,date_before,errandNumber_after,date_after,insuranceNumber_consistant,damageNumber_consistant,invoiceReference_consistant
0,Jenna,Agneta Nyström,If,16220687095,2025-06-09 13:54:56.335324+00:00,16220689836,2025-06-12 08:07:48.508955+00:00,True,True,False
1,Mini,Anders Eriksson,Agria,39433750658,2025-09-01 12:46:07.492328+00:00,39433750989,2025-09-02 07:19:39.908932+00:00,True,False,False
2,Hilja,Anders Svahn,Folksam,38568730311,2025-08-06 07:56:10.429513+00:00,38568732408,2025-08-08 07:04:51.851619+00:00,True,True,False
3,Hilja,Anders Svahn,Folksam,38568730311,2025-08-06 07:56:10.429513+00:00,38568738041,2025-08-15 12:45:44.422810+00:00,True,True,False
4,Hilja,Anders Svahn,Folksam,38568730311,2025-08-06 07:56:10.429513+00:00,38568738595,2025-08-18 09:09:54.390669+00:00,True,True,False


In [101]:
e['återbesök'] = [[] for i in range(len(e))] 
f = e.groupby(['animalName','ownerName','insuranceCompany','errandNumber_before'])
# for idx, r in f:
#     if r.errandNumber_before.count()>2:
#         print("=========== ",idx," =============\n",r)

In [102]:
g = e[['animalName','ownerName','insuranceCompany','errandNumber_before']]
g = g.drop_duplicates()
g.errandNumber_before.count()

np.int64(106)

In [103]:
query = '''SELECT
    c.name AS "Klinik",
    prev.reference AS "Ärende1",
    prev."createdAt" AS "date1",
    e1.reference AS "Ärende2",
    e1."createdAt" AS "date2"
    FROM clinic c
    JOIN errand e1 ON e1."clinicId" = c.id
    JOIN insurance_case ic1 ON ic1."errandId" = e1.id
    JOIN insurance_company_email ice1 ON ic1."insuranceCompanyEmailId" = ice1.id
    INNER JOIN LATERAL (
    SELECT e2.reference, e2."createdAt"
    FROM insurance_case ic2
    JOIN insurance_company_email ice2 ON ic2."insuranceCompanyEmailId" = ice2.id
    JOIN errand e2 ON ic2."errandId" = e2.id
    JOIN errand_status es ON es.id = e2."statusId"
    WHERE ic2."insuranceNumber" = ic1."insuranceNumber"
        AND ice2."insuranceCompanyId" = ice1."insuranceCompanyId"
        AND e2.id <> e1.id
        AND e2."clinicId" = e1."clinicId"
        AND e2."createdAt" <= e1."createdAt"
        AND e2."createdAt" >= (e1."createdAt" - INTERVAL '5 days')
        AND es."cancelled" IS FALSE
    ORDER BY e2."createdAt" DESC   -- closest prior
    LIMIT 1
    ) AS prev ON TRUE
    WHERE c.id = 38
    AND EXTRACT(YEAR FROM e1."createdAt") = 2025;'''
emy = fetchFromDB(query)
emy

,Klinik,Ärende1,date1,Ärende2,date2
0,Hälsinge Smådjursklinik Hudiksvall,32970599398,2025-02-07 13:53:12.700907+00:00,32970599760,2025-02-08 08:52:45.665570+00:00
1,Hälsinge Smådjursklinik Hudiksvall,16203595356,2025-02-04 10:01:11.796588+00:00,16203597279,2025-02-05 15:06:26.356796+00:00
2,Hälsinge Smådjursklinik Hudiksvall,25301588445,2025-01-27 09:58:29.341031+00:00,25301592561,2025-01-30 16:17:24.820264+00:00
3,Hälsinge Smådjursklinik Hudiksvall,31836571005,2025-01-02 14:52:16.571501+00:00,31836572227,2025-01-05 11:35:28.560724+00:00
4,Hälsinge Smådjursklinik Hudiksvall,32804594235,2025-02-03 10:57:57.780642+00:00,32804596065,2025-02-04 14:21:34.175280+00:00
...,...,...,...,...,...
133,Hälsinge Smådjursklinik Hudiksvall,39433750658,2025-09-01 12:46:07.492328+00:00,39433750989,2025-09-02 07:19:39.908932+00:00
134,Hälsinge Smådjursklinik Hudiksvall,39489751964,2025-09-02 14:39:31.892744+00:00,39489752095,2025-09-03 07:35:50.378746+00:00
135,Hälsinge Smådjursklinik Hudiksvall,39401750896,2025-09-01 15:05:06.010931+00:00,39401752371,2025-09-03 09:42:59.176342+00:00
136,Hälsinge Smådjursklinik Hudiksvall,39159753282,2025-09-04 08:59:42.567744+00:00,39159753308,2025-09-04 09:19:32.768594+00:00


In [106]:
h = emy.merge(e[['errandNumber_before', 'errandNumber_after']], left_on=['Ärende1', 'Ärende2'], right_on=['errandNumber_before', 'errandNumber_after'], how='left')
h.loc[h['Ärende1'].notna() & h['errandNumber_before'].isna()].sort_values('Ärende1').head()

,Klinik,Ärende1,date1,Ärende2,date2,errandNumber_before,errandNumber_after
20,Hälsinge Smådjursklinik Hudiksvall,13249583660,2025-01-20 15:42:24.710690+00:00,13249583991,2025-01-21 09:58:18.168951+00:00,NaN,NaN
137,Hälsinge Smådjursklinik Hudiksvall,13585750204,2025-09-01 09:36:32.337658+00:00,13585754404,2025-09-05 07:49:11.746104+00:00,NaN,NaN
30,Hälsinge Smådjursklinik Hudiksvall,15133615474,2025-02-28 13:15:58.916833+00:00,15133616381,2025-03-03 11:06:54.664510+00:00,NaN,NaN
34,Hälsinge Smådjursklinik Hudiksvall,15133616381,2025-03-03 11:06:54.664510+00:00,15133619104,2025-03-05 14:53:56.031371+00:00,NaN,NaN
33,Hälsinge Smådjursklinik Hudiksvall,15133619104,2025-03-05 14:53:56.031371+00:00,15133619344,2025-03-06 08:49:15.429581+00:00,NaN,NaN


In [107]:
import pandas as pd

def merge_chain(df: pd.DataFrame, col1: str, col2: str) -> pd.DataFrame:
    df = df.copy()
    result = []
    start_nodes = set(df[col1]) - set(df[col2])
    for start in start_nodes:
        chain = []
        current = start
        while True:
            row = df.loc[df[col1] == current]
            if row.empty:
                break
            next_val = row[col2].values[0]
            chain.append(next_val)
            current = next_val
        result.append({col1: start, col2: chain})
    return pd.DataFrame(result)

sub = merge_chain(emy, 'Ärende1','Ärende2')
sub

,Ärende1,Ärende2
0,25895600143,[25895603949]
1,25301585904,"[25301588445, 25301592561]"
2,35918673704,"[35918675147, 35918677861]"
3,33076600684,[33076604025]
4,20380645281,[20380645935]
...,...,...
106,34273630219,[34273630235]
107,39205744988,[39205748344]
108,32970599398,[32970599760]
109,33736617037,[33736617300]


In [108]:
import pandas as pd
from typing import List, Optional

def compress_errand_sequences(
    df: pd.DataFrame,
    number_col: str = "errandNumber",
    label_col: str = "gap_label",
    id_col: Optional[str] = None,
    keep_extra_cols: Optional[List[str]] = None,
) -> pd.DataFrame:
    """
    将按序排列的链式多行压成一行：
      - 每个序列以 gap_label == 'first' 开始
      - 该序列后续行的 errandNumber 收集到 subsequence 列
      - 如果有多条链（多个 'first'），各自输出一行

    参数
    ----
    df : 输入DataFrame（应按时间/链顺序排好）
    number_col : 存放案件号的列名
    label_col : 存放标签（如 'first', '3day...'）的列名
    id_col : 可选，若存在则保留第一行该列
    keep_extra_cols : 可选，额外需要从第一行保留的列名列表

    返回
    ----
    DataFrame：列包含
      - 如果给了 id_col：该列
      - number_col：第一行的案件号
      - label_col：'first'
      - subsequence：list[str]，后续行的案件号（字符串形式）
    """
    if keep_extra_cols is None:
        keep_extra_cols = []

    required = [number_col, label_col]
    missing = [c for c in required if c not in df.columns]
    if missing:
        raise ValueError(f"Missing required columns: {missing}")

    records = []
    current_group = None

    def flush_group():
        nonlocal current_group
        if current_group is not None:
            records.append(current_group)
            current_group = None

    # 逐行扫描，遇到 'first' 开启新组，其余加入 subsequence
    for _, row in df.iterrows():
        lbl = row[label_col]
        num = row[number_col]

        if lbl == "first":
            # 先把前一个组收尾
            flush_group()

            # 新组：以当前行为主行
            current_group = {
                number_col: num,
                label_col: "first",
                "subsequence": [],  # 后续行会填进来
            }
            if id_col and id_col in df.columns:
                current_group[id_col] = row[id_col]
            for c in keep_extra_cols:
                if c in df.columns:
                    current_group[c] = row[c]
        else:
            # 非 first：挂到当前组的 subsequence
            if current_group is None:
                # 若数据不规范（第一行就不是 first），则开启匿名组
                current_group = {
                    number_col: num,
                    label_col: lbl,
                    "subsequence": [],
                }
                if id_col and id_col in df.columns:
                    current_group[id_col] = row.get(id_col)
                for c in keep_extra_cols:
                    if c in df.columns:
                        current_group[c] = row[c]
            else:
                current_group["subsequence"].append(str(num))

    # 收尾
    flush_group()

    # 规范输出列顺序
    cols = []
    if id_col and (len(records) == 0 or id_col in records[0]):
        cols.append(id_col)
    cols += [number_col, label_col, "subsequence"]
    # 附加保留列（若有）
    for c in keep_extra_cols:
        if len(records) and c in records[0] and c not in cols:
            cols.append(c)

    out = pd.DataFrame(records)
    # 只保留存在的列，避免 KeyError
    cols = [c for c in cols if c in out.columns]
    return out[cols]
j = compress_errand_sequences(filtered)
j.head()

,errandNumber,gap_label,subsequence
0,16220687095,first,[16220689836]
1,39433750658,first,[39433750989]
2,38568730311,first,[38568732408]
3,38568735666,first,"[38568738041, 38568738595]"
4,33736617037,first,[33736617300]


In [109]:
g = sub.merge(j, left_on='Ärende1', right_on='errandNumber', how='left')
g

,Ärende1,Ärende2,errandNumber,gap_label,subsequence
0,25895600143,[25895603949],25895600143,first,[25895603949]
1,25301585904,"[25301588445, 25301592561]",25301585904,first,"[25301588445, 25301592561]"
2,35918673704,"[35918675147, 35918677861]",35918673704,first,"[35918675147, 35918677861]"
3,33076600684,[33076604025],33076600684,first,[33076604025]
4,20380645281,[20380645935],20380645281,first,[20380645935]
...,...,...,...,...,...
106,34273630219,[34273630235],34273630219,first,[34273630235]
107,39205744988,[39205748344],39205744988,first,[39205748344]
108,32970599398,[32970599760],32970599398,first,[32970599760]
109,33736617037,[33736617300],33736617037,first,[33736617300]


In [110]:
for idx, row in g.iterrows():
    if row['Ärende2'] == row['subsequence']:
        g.at[idx,'eq'] = 'EQ'
g.loc[g['errandNumber'].isna()]

,Ärende1,Ärende2,errandNumber,gap_label,subsequence,eq
5,31727569761,[31727571528],NaN,NaN,NaN,NaN
31,39240745982,"[24064747736, 24064748940]",NaN,NaN,NaN,NaN
41,31770569445,[31770571995],NaN,NaN,NaN,NaN
60,37482706019,[37482706027],NaN,NaN,NaN,NaN
95,35187652082,[35187655044],NaN,NaN,NaN,NaN
100,34364632322,[34364634138],NaN,NaN,NaN,NaN


In [90]:
er.loc[er['errandNumber']=='31727569761']

,errandId,errandNumber,date,insuranceCompany,clinicName,totalAmount,settlementAmount,reference,insuranceNumber,damageNumber,invoiceReference,animalName,ownerName,paymentOption,strategyType,settled,createdAt


In [116]:
query = '''WITH
  errands_2025 AS (
    SELECT
      er.id,
      er.reference,
      er."createdAt",
      er."animalOwnerId",
      ic."insuranceNumber",
      ic."animalId"
    FROM errand er
    JOIN insurance_case ic ON ic."errandId" = er.id
    JOIN errand_status es ON es.id = er."statusId"
    WHERE
      er."clinicId" = 38
      AND er."createdAt" >= '2025-01-01'
      AND er."createdAt" < '2026-01-01'
      AND ic."insuranceNumber" IS NOT NULL
      AND ic."insuranceNumber" <> ''
      AND es."cancelled" IS FALSE
  ),
  paired_errands AS (
    SELECT
      e1."animalOwnerId",
      e1."insuranceNumber",
      e2.reference AS "errandNumber1",
      e2."createdAt" AS "date1",
      e1.reference AS "errandNumber2",
      e1."createdAt" AS "date2",
      e1."animalId"
    FROM
      errands_2025 e1
      JOIN errands_2025 e2 ON e1."animalOwnerId" = e2."animalOwnerId"
      AND e1."insuranceNumber" = e2."insuranceNumber"
      AND e1.id <> e2.id
      AND e1."createdAt" > e2."createdAt"
      AND e1."createdAt" <= e2."createdAt" + INTERVAL '5 days'
  )
    SELECT DISTINCT p."errandNumber1", p."date1", p."errandNumber2", p."date2",
    a.name AS "animalName",
    CONCAT(ao."firstName", ' ', ao."lastName") AS "ownerName",
    p."insuranceNumber"
    FROM paired_errands p
    JOIN animal a ON a.id = p."animalId"
    JOIN animal_owner ao ON ao.id = p."animalOwnerId"
    ORDER BY p."errandNumber1";'''
line = fetchFromDB(query)
l = merge_chain(line, 'errandNumber1', 'errandNumber2')
l

,errandNumber1,errandNumber2
0,25895600143,[25895603949]
1,25301585904,"[25301588445, 25301592561]"
2,35918673704,"[35918675147, 35918677861]"
3,33076600684,[33076604025]
4,20380645281,[20380645935]
...,...,...
103,34273630219,[34273630235]
104,39205744988,[39205748344]
105,32970599398,[32970599760]
106,33736617037,[33736617300]


In [128]:
l.loc[l['errandNumber2'].apply(len)>1]

,errandNumber1,errandNumber2
1,25301585904,"[25301588445, 25301592561]"
2,35918673704,"[35918675147, 35918677861]"
6,38153725148,"[38153727391, 38153728829, 38153729835]"
21,35842666782,"[35842668085, 35842668945]"
24,22827574983,"[22827576814, 22827580493]"
29,15133614394,"[15133615474, 15133616381, 15133619104, 151336..."
40,35721664072,"[35721665301, 35721666770]"
44,38841736317,"[38841737315, 38841737547]"
45,36237676063,"[36237677095, 36237677202]"
53,35208658704,"[35208660106, 35208662847]"


In [132]:
l.loc[l['errandNumber1']=='15133614394','errandNumber2'].to_list()

[['15133615474', '15133616381', '15133619104', '15133619344', '15133622488']]

In [125]:
g = l.merge(sub, left_on='errandNumber1', right_on='Ärende1', how='left')
g

,errandNumber1,errandNumber2,Ärende1,Ärende2
0,25895600143,[25895603949],25895600143,[25895603949]
1,25301585904,"[25301588445, 25301592561]",25301585904,"[25301588445, 25301592561]"
2,35918673704,"[35918675147, 35918677861]",35918673704,"[35918675147, 35918677861]"
3,33076600684,[33076604025],33076600684,[33076604025]
4,20380645281,[20380645935],20380645281,[20380645935]
...,...,...,...,...
103,34273630219,[34273630235],34273630219,[34273630235]
104,39205744988,[39205748344],39205744988,[39205748344]
105,32970599398,[32970599760],32970599398,[32970599760]
106,33736617037,[33736617300],33736617037,[33736617300]


### summary test

In [ ]:
from app.dataset.summary_dataset import SummaryDataset
import pandas as pd

email_id = 168623
reference = None
errand_number = None
summary_df = pd.DataFrame([{
            'emailId':email_id,
            'errandNumber':errand_number,
            'reference':reference}])
ds = SummaryDataset(df=summary_df)
a = ds.do_summary()
a

,summary_combined
0,* FB meddelar att ärendet med referens journal...


### log testing

In [ ]:
from app.dataset.log_dataset import LogDataset
import pandas as pd

errand_number = '39863761266'
errand_df = pd.DataFrame([{'errand_number': errand_number}])
ds = LogDataset(df=errand_df)
a = ds.do_chronological_log()
a

,errand_id,log_title,log_content,ai_analysis,error_message
0,76126,Ärenden: 76126,"<br><span style=""color:blue;"">2025-09-15</span...","<div style=""text-align: center;"">*************...",None


### read Payex invoice

In [ ]:
import pandas as pd
from app.services.utils import fetchFromDB

query = ''' SELECT DISTINCT 
                er.id AS "errandId", 
                pif."fileName" ,
                er."createdAt" ,
                er."paymentOption"
            FROM "transaction" t 
            JOIN errand er ON er.id = t."errandId"
            JOIN payex_invoice pyi ON t.id = pyi."customerInvoiceId" 
            JOIN payex_invoice_file pif ON pyi."payexInvoiceFileId" = pif.id
            WHERE er."paymentOption" = 'payex'
              AND t.type_ = 'customer_invoice'
              AND er.reference = '9619127450'
            ORDER BY er."createdAt" DESC;'''
df = fetchFromDB(query)
print(df)
file_path = f"gs://drp-system-production/payex/incoming/{df['fileName'].iloc[0]}"
print(file_path)
payex_invoice_df = pd.read_xml(file_path) 
payex_invoice_df                    

In [5]:
# First, let's install the required Google Cloud Storage library if not already installed
import subprocess
import sys

try:
    from google.cloud import storage
    print("google-cloud-storage is already installed")
except ImportError:
    print("Installing google-cloud-storage...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "google-cloud-storage"])
    from google.cloud import storage
    print("google-cloud-storage installed successfully")

google-cloud-storage is already installed


In [ ]:
import os
from google.cloud import storage
import pandas as pd

# Set the service account key file path
service_account_path = "/data/other/drp-system-73cd3f0ca038.json"

# Set the environment variable for authentication
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = service_account_path

# Initialize the storage client
client = storage.Client()

print(f"Authenticated with service account: {client.project}")
print("Available buckets:")
for bucket in client.list_buckets():
    print(f"  - {bucket.name}")

In [8]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'data/other/drp-system-73cd3f0ca038.json'

import pandas as pd

file_path = "gs://drp-system-production/payex/incoming/CUSIN_24721_20230714110000_149.xml"

try:
    payex_invoice_df = pd.read_xml(file_path)
    print("Successfully read XML file from GCS!")
    print(f"DataFrame shape: {payex_invoice_df.shape}")
    print("\nFirst few rows:")
    print(payex_invoice_df.head())
except Exception as e:
    print(f"Error reading XML file: {e}")
    print("\nLet's try using Google Cloud Storage client instead...")

_request out of retries on exception: ('invalid_grant: Bad Request', {'error': 'invalid_grant', 'error_description': 'Bad Request'})
Traceback (most recent call last):
  File "/Users/jiechen/Apoex/Email/Email_Execution_Tree/fastAPI/.venv/lib/python3.13/site-packages/gcsfs/retry.py", line 130, in retry_request
    return await func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jiechen/Apoex/Email/Email_Execution_Tree/fastAPI/.venv/lib/python3.13/site-packages/gcsfs/core.py", line 431, in _request
    headers=self._get_headers(headers),
            ~~~~~~~~~~~~~~~~~^^^^^^^^^
  File "/Users/jiechen/Apoex/Email/Email_Execution_Tree/fastAPI/.venv/lib/python3.13/site-packages/gcsfs/core.py", line 408, in _get_headers
    self.credentials.apply(out)
    ~~~~~~~~~~~~~~~~~~~~~~^^^^^
  File "/Users/jiechen/Apoex/Email/Email_Execution_Tree/fastAPI/.venv/lib/python3.13/site-packages/gcsfs/credentials.py", line 188, in apply
    self.maybe_refresh()
    ~~~~~~~~~~~~~~~~~~^^

Error reading XML file: ('invalid_grant: Bad Request', {'error': 'invalid_grant', 'error_description': 'Bad Request'})

Let's try using Google Cloud Storage client instead...


In [9]:
# Let's check if the service account file exists and fix the path
import os
import json

# Check the current working directory
print(f"Current working directory: {os.getcwd()}")

# Try the absolute path first
service_account_path = "/Users/jiechen/Apoex/Email/Email_Execution_Tree/fastAPI/data/other/drp-system-73cd3f0ca038.json"
print(f"\nChecking absolute path: {service_account_path}")
print(f"File exists: {os.path.exists(service_account_path)}")

if os.path.exists(service_account_path):
    # Try to read and validate the JSON file
    try:
        with open(service_account_path, 'r') as f:
            credentials = json.load(f)
        print("Service account file is valid JSON")
        print(f"Project ID: {credentials.get('project_id', 'Not found')}")
        print(f"Client email: {credentials.get('client_email', 'Not found')}")
        print(f"Type: {credentials.get('type', 'Not found')}")
    except Exception as e:
        print(f"Error reading service account file: {e}")
else:
    # Try relative path
    relative_path = "data/other/drp-system-73cd3f0ca038.json"
    print(f"\nTrying relative path: {relative_path}")
    print(f"File exists: {os.path.exists(relative_path)}")
    
    if os.path.exists(relative_path):
        service_account_path = relative_path

Current working directory: /Users/jiechen/Apoex/Email/Email_Execution_Tree/fastAPI

Checking absolute path: /Users/jiechen/Apoex/Email/Email_Execution_Tree/fastAPI/data/other/drp-system-73cd3f0ca038.json
File exists: True
Service account file is valid JSON
Project ID: drp-system
Client email: drp-test@drp-system.iam.gserviceaccount.com
Type: service_account


In [10]:
# Set up authentication properly and try to read the XML file
import os
from google.cloud import storage
import pandas as pd

# Set the correct path
service_account_path = "/Users/jiechen/Apoex/Email/Email_Execution_Tree/fastAPI/data/other/drp-system-73cd3f0ca038.json"
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = service_account_path

try:
    # Initialize the storage client
    client = storage.Client()
    
    # Get the bucket and blob
    bucket_name = "drp-system-production"
    blob_name = "payex/incoming/CUSIN_24721_20230714110000_149.xml"
    
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(blob_name)
    
    # Download the content as string
    xml_content = blob.download_as_text()
    
    # Save to a temporary file and read with pandas
    import tempfile
    with tempfile.NamedTemporaryFile(mode='w', suffix='.xml', delete=False) as temp_file:
        temp_file.write(xml_content)
        temp_file_path = temp_file.name
    
    # Now read the XML file with pandas
    payex_invoice_df = pd.read_xml(temp_file_path)
    
    # Clean up temp file
    os.unlink(temp_file_path)
    
    print("Successfully read XML file from GCS!")
    print(f"DataFrame shape: {payex_invoice_df.shape}")
    print("\nColumns:", payex_invoice_df.columns.tolist())
    print("\nFirst few rows:")
    print(payex_invoice_df.head())
    
except Exception as e:
    print(f"Error: {e}")
    print("Let's try listing available files in the bucket first...")

Successfully read XML file from GCS!
DataFrame shape: (7, 4)

Columns: ['AsSoonAsPossible', 'CustomerNo', 'CustomerInfo', 'Invoices']

First few rows:
   AsSoonAsPossible  CustomerNo CustomerInfo  Invoices
0               NaN         NaN         None      None
1               NaN      6892.0     \n        \n      
2               NaN      8613.0     \n        \n      
3               NaN      8614.0     \n        \n      
4               NaN      8617.0     \n        \n      


In [11]:
payex_invoice_df

,AsSoonAsPossible,CustomerNo,CustomerInfo,Invoices
0,NaN,NaN,None,None
1,NaN,6892.0,\n,\n
2,NaN,8613.0,\n,\n
3,NaN,8614.0,\n,\n
4,NaN,8617.0,\n,\n
5,NaN,8619.0,\n,\n
6,NaN,9286.0,\n,\n


In [12]:
# Method 1: Read and display the raw XML content
with open('/Users/jiechen/Apoex/Email/Email_Execution_Tree/fastAPI/data/test_data/1.xml', 'r') as f:
    xml_content = f.read()

print("Raw XML content (first 1000 characters):")
print(xml_content[:1000])
print("..." if len(xml_content) > 1000 else "")

Raw XML content (first 1000 characters):
This XML file does not appear to have any style information associated with it. The document tree is shown below.
<CUSIN CompanyName="Apoex Systems" CompanyNo="24721" CreatedDateTime="2023-07-14T09:00:00.399Z" Version="2.0">
<DistributionDate>
<AsSoonAsPossible/>
</DistributionDate>
<CUSINInfo>
<CustomerNo>6892</CustomerNo>
<CustomerInfo>
<Name>Camilla Vanhatalo</Name>
<RegNo CountryCode="SE">19690503-8920</RegNo>
<DistributionAddressInfo>
<Addresses>
<Address AddressType="LegalAddress">
<Addressee>Camilla Vanhatalo</Addressee>
<StreetAddress>Schouggatan 15 lgh 1103</StreetAddress>
<ZipCode>98136</ZipCode>
<City>Kiruna</City>
<CoAddress/>
<CountryCode>SE</CountryCode>
</Address>
<Address AddressType="BillingAddress">
<Addressee>Camilla Vanhatalo</Addressee>
<StreetAddress>Schouggatan 15 lgh 1103</StreetAddress>
<ZipCode>98136</ZipCode>
<City>Kiruna</City>
<CoAddress/>
<CountryCode>SE</CountryCode>
</Address>
</Addresses>
<E-MailAddress>cvanhatal

In [13]:
# Method 2: Pretty print the XML with proper formatting
import xml.etree.ElementTree as ET
from xml.dom import minidom

def pretty_print_xml(file_path, max_lines=50):
    """Pretty print XML file with formatting"""
    try:
        # Parse the XML
        tree = ET.parse(file_path)
        root = tree.getroot()
        
        # Convert to string with pretty formatting
        rough_string = ET.tostring(root, 'unicode')
        reparsed = minidom.parseString(rough_string)
        pretty_xml = reparsed.toprettyxml(indent="  ")
        
        # Split into lines and show first max_lines
        lines = pretty_xml.split('\n')
        lines = [line for line in lines if line.strip()]  # Remove empty lines
        
        print(f"XML Structure (showing first {min(len(lines), max_lines)} lines):")
        print("-" * 60)
        for i, line in enumerate(lines[:max_lines]):
            print(f"{i+1:3d}: {line}")
        
        if len(lines) > max_lines:
            print(f"... ({len(lines) - max_lines} more lines)")
            
        return len(lines)
    except Exception as e:
        print(f"Error parsing XML: {e}")
        return 0

total_lines = pretty_print_xml('/Users/jiechen/Apoex/Email/Email_Execution_Tree/fastAPI/data/test_data/1.xml')

Error parsing XML: syntax error: line 1, column 0


In [14]:
# Method 3: Read with pandas and show structure
import pandas as pd

try:
    # Try to read as XML with pandas
    df = pd.read_xml('/Users/jiechen/Apoex/Email/Email_Execution_Tree/fastAPI/data/test_data/1.xml')
    print("XML successfully parsed with pandas!")
    print(f"DataFrame shape: {df.shape}")
    print(f"Columns: {df.columns.tolist()}")
    print("\nFirst few rows:")
    print(df.head())
except Exception as e:
    print(f"Pandas XML parsing error: {e}")
    
    # Method 4: Clean the XML content and try again
    print("\nTrying to clean XML content...")
    
    with open('/Users/jiechen/Apoex/Email/Email_Execution_Tree/fastAPI/data/test_data/1.xml', 'r') as f:
        content = f.read()
    
    # Remove the style information comment if present
    if "This XML file does not appear to have any style information" in content:
        # Find the actual XML start
        xml_start = content.find('<CUSIN')
        if xml_start != -1:
            clean_content = content[xml_start:]
            print(f"Found XML content starting at position {xml_start}")
            print("Clean XML content (first 500 chars):")
            print(clean_content[:500])
            print("...")
        else:
            print("Could not find XML root element")
    else:
        print("XML appears to be clean already")

Pandas XML parsing error: Start tag expected, '<' not found, line 1, column 1 (<string>, line 1)

Trying to clean XML content...
Found XML content starting at position 114
Clean XML content (first 500 chars):
<CUSIN CompanyName="Apoex Systems" CompanyNo="24721" CreatedDateTime="2023-07-14T09:00:00.399Z" Version="2.0">
<DistributionDate>
<AsSoonAsPossible/>
</DistributionDate>
<CUSINInfo>
<CustomerNo>6892</CustomerNo>
<CustomerInfo>
<Name>Camilla Vanhatalo</Name>
<RegNo CountryCode="SE">19690503-8920</RegNo>
<DistributionAddressInfo>
<Addresses>
<Address AddressType="LegalAddress">
<Addressee>Camilla Vanhatalo</Addressee>
<StreetAddress>Schouggatan 15 lgh 1103</StreetAddress>
<ZipCode>98136</ZipCode>

...


In [15]:
# Method 5: Clean the XML and parse it properly
import tempfile
import xml.etree.ElementTree as ET

# Read and clean the XML file
with open('/Users/jiechen/Apoex/Email/Email_Execution_Tree/fastAPI/data/test_data/1.xml', 'r') as f:
    content = f.read()

# Remove the comment at the beginning
xml_start = content.find('<CUSIN')
if xml_start != -1:
    clean_xml = content[xml_start:]
    
    # Save clean XML to temp file
    with tempfile.NamedTemporaryFile(mode='w', suffix='.xml', delete=False) as temp_file:
        temp_file.write(clean_xml)
        temp_path = temp_file.name
    
    try:
        # Parse with pandas
        df = pd.read_xml(temp_path)
        print("✅ Successfully parsed XML with pandas!")
        print(f"DataFrame shape: {df.shape}")
        print(f"Columns: {df.columns.tolist()}")
        print("\nDataFrame preview:")
        print(df.head())
        
    except Exception as e:
        print(f"Pandas still failed: {e}")
        
        # Try with ElementTree for structure analysis
        try:
            tree = ET.parse(temp_path)
            root = tree.getroot()
            
            print(f"\n✅ Successfully parsed with ElementTree!")
            print(f"Root element: {root.tag}")
            print(f"Root attributes: {root.attrib}")
            
            # Show child elements
            print("\nDirect child elements:")
            for i, child in enumerate(root):
                print(f"  {i+1}: {child.tag} - {len(list(child))} children")
                if i < 3:  # Show first few children in detail
                    for j, grandchild in enumerate(child):
                        if j < 3:
                            print(f"    - {grandchild.tag}: {grandchild.text if grandchild.text else 'has children'}")
                        elif j == 3:
                            print(f"    - ... ({len(list(child))-3} more)")
                            break
            
        except Exception as e2:
            print(f"ElementTree also failed: {e2}")
    
    # Clean up
    import os
    os.unlink(temp_path)

✅ Successfully parsed XML with pandas!
DataFrame shape: (7, 4)
Columns: ['AsSoonAsPossible', 'CustomerNo', 'CustomerInfo', 'Invoices']

DataFrame preview:
   AsSoonAsPossible  CustomerNo CustomerInfo Invoices
0               NaN         NaN         None     None
1               NaN      6892.0           \n       \n
2               NaN      8613.0           \n       \n
3               NaN      8614.0           \n       \n
4               NaN      8617.0           \n       \n


In [16]:
# Complete XML preview function you can reuse
def preview_xml_file(file_path, max_rows=10):
    """Preview an XML file, handling common formatting issues"""
    import pandas as pd
    import tempfile
    import os
    
    print(f"📄 Previewing XML file: {file_path}")
    print("=" * 60)
    
    # Read the file
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
    
    print(f"File size: {len(content)} characters")
    
    # Check for and remove common XML comments/headers
    xml_start = content.find('<')
    if xml_start > 0:
        print(f"⚠️  Found {xml_start} characters before XML content, cleaning...")
        # Find the actual XML root element
        possible_roots = ['<CUSIN', '<root', '<xml', '<?xml']
        xml_start = -1
        for root_pattern in possible_roots:
            pos = content.find(root_pattern)
            if pos != -1:
                xml_start = pos
                break
        
        if xml_start != -1:
            content = content[xml_start:]
    
    # Create temp file with clean content
    with tempfile.NamedTemporaryFile(mode='w', suffix='.xml', delete=False, encoding='utf-8') as temp_file:
        temp_file.write(content)
        temp_path = temp_file.name
    
    try:
        # Parse with pandas
        df = pd.read_xml(temp_path)
        print(f"✅ Successfully parsed with pandas!")
        print(f"DataFrame shape: {df.shape}")
        print(f"Columns: {list(df.columns)}")
        
        if len(df) > 0:
            print(f"\nFirst {min(max_rows, len(df))} rows:")
            print(df.head(max_rows))
        
        return df
        
    except Exception as e:
        print(f"❌ Pandas parsing failed: {e}")
        print("Raw XML preview (first 500 chars):")
        print(content[:500])
        return None
    
    finally:
        # Cleanup
        os.unlink(temp_path)

# Test it with your file
df = preview_xml_file('/Users/jiechen/Apoex/Email/Email_Execution_Tree/fastAPI/data/test_data/1.xml')

📄 Previewing XML file: /Users/jiechen/Apoex/Email/Email_Execution_Tree/fastAPI/data/test_data/1.xml
File size: 30894 characters
⚠️  Found 114 characters before XML content, cleaning...
✅ Successfully parsed with pandas!
DataFrame shape: (7, 4)
Columns: ['AsSoonAsPossible', 'CustomerNo', 'CustomerInfo', 'Invoices']

First 7 rows:
   AsSoonAsPossible  CustomerNo CustomerInfo Invoices
0               NaN         NaN         None     None
1               NaN      6892.0           \n       \n
2               NaN      8613.0           \n       \n
3               NaN      8614.0           \n       \n
4               NaN      8617.0           \n       \n
5               NaN      8619.0           \n       \n
6               NaN      9286.0           \n       \n


In [17]:
# Let's examine the XML structure in detail to find 'Att betala (SEK)'
import xml.etree.ElementTree as ET
import tempfile

# Read and clean the XML
with open('/Users/jiechen/Apoex/Email/Email_Execution_Tree/fastAPI/data/test_data/1.xml', 'r') as f:
    content = f.read()

xml_start = content.find('<CUSIN')
clean_xml = content[xml_start:]

# Save to temp file
with tempfile.NamedTemporaryFile(mode='w', suffix='.xml', delete=False) as temp_file:
    temp_file.write(clean_xml)
    temp_path = temp_file.name

def explore_xml_structure(element, level=0, max_level=4):
    """Recursively explore XML structure"""
    indent = "  " * level
    print(f"{indent}{element.tag}")
    
    # Print attributes if any
    if element.attrib:
        print(f"{indent}  @attributes: {element.attrib}")
    
    # Print text content if it's not just whitespace
    if element.text and element.text.strip():
        text = element.text.strip()
        if len(text) > 50:
            text = text[:50] + "..."
        print(f"{indent}  text: '{text}'")
    
    # Look for the specific column we're interested in
    if 'betala' in element.tag.lower() or 'sek' in element.tag.lower():
        print(f"{indent}  ⭐ FOUND RELEVANT: {element.tag} = {element.text}")
    
    # Recursively explore children if not too deep
    if level < max_level:
        for child in element:
            explore_xml_structure(child, level + 1, max_level)

# Parse and explore
tree = ET.parse(temp_path)
root = tree.getroot()

print("🔍 XML Structure Analysis:")
print("=" * 60)
explore_xml_structure(root)

# Cleanup
import os
os.unlink(temp_path)

🔍 XML Structure Analysis:
CUSIN
  @attributes: {'CompanyName': 'Apoex Systems', 'CompanyNo': '24721', 'CreatedDateTime': '2023-07-14T09:00:00.399Z', 'Version': '2.0'}
  DistributionDate
    AsSoonAsPossible
  CUSINInfo
    CustomerNo
      text: '6892'
    CustomerInfo
      Name
        text: 'Camilla Vanhatalo'
      RegNo
        @attributes: {'CountryCode': 'SE'}
        text: '19690503-8920'
      DistributionAddressInfo
        Addresses
        E-MailAddress
          text: 'cvanhatalo@gmail.com'
      PaymentInfo
        Set
      DistributionInfo
        Set
    Invoices
      InvoiceWithDistribution
        InvoiceNo
          text: '667'
        InvoiceDate
          text: '2023-07-14'
        PreferredDueDate
          text: '2023-08-13'
        Amount
          @attributes: {'Currency': 'SEK'}
        DistributionInfo
  CUSINInfo
    CustomerNo
      text: '8613'
    CustomerInfo
      Name
        text: 'Daria Kowalewska'
      RegNo
        @attributes: {'CountryCode': '

In [18]:
# Let's search for the specific text and try different parsing approaches
import re

# Search for the specific column name in the raw content
with open('/Users/jiechen/Apoex/Email/Email_Execution_Tree/fastAPI/data/test_data/1.xml', 'r') as f:
    content = f.read()

# Look for the specific text
if 'Att betala (SEK)' in content:
    print("✅ Found 'Att betala (SEK)' in the file!")
    
    # Find all occurrences and their context
    matches = []
    start = 0
    while True:
        pos = content.find('Att betala (SEK)', start)
        if pos == -1:
            break
        # Get some context around the match
        context_start = max(0, pos - 100)
        context_end = min(len(content), pos + 200)
        context = content[context_start:context_end]
        matches.append((pos, context))
        start = pos + 1
    
    print(f"Found {len(matches)} occurrences:")
    for i, (pos, context) in enumerate(matches[:3]):  # Show first 3
        print(f"\n--- Match {i+1} at position {pos} ---")
        print(context)
else:
    print("❌ 'Att betala (SEK)' not found in raw content")
    
    # Let's search for variations
    patterns = ['betala', 'SEK', 'Amount', 'Total', 'Sum']
    print("\nSearching for related terms:")
    for pattern in patterns:
        count = content.lower().count(pattern.lower())
        if count > 0:
            print(f"  '{pattern}': {count} occurrences")
            
            # Show first occurrence context
            pos = content.lower().find(pattern.lower())
            if pos != -1:
                context_start = max(0, pos - 50)
                context_end = min(len(content), pos + 100)
                context = content[context_start:context_end]
                print(f"    Context: ...{context}...")
            print()

✅ Found 'Att betala (SEK)' in the file!
Found 6 occurrences:

--- Match 1 at position 5603 ---
o="3">
<Text>2434,40</Text>
</Column>
</Columns>
</Row>
<Row>
<Columns>
<Column ColumnNo="2">
<Text>Att betala (SEK)</Text>
</Column>
<Column ColumnNo="3">
<Text>3043,00</Text>
</Column>
</Columns>
</Row>
</Rows>
</SubOrderBlock>
</SubOrderBlocks>
</BasePrintBlock>
</StandardPrintBlock>
</InvoicePri

--- Match 2 at position 10748 ---
o="3">
<Text>2525,60</Text>
</Column>
</Columns>
</Row>
<Row>
<Columns>
<Column ColumnNo="2">
<Text>Att betala (SEK)</Text>
</Column>
<Column ColumnNo="3">
<Text>3157,00</Text>
</Column>
</Columns>
</Row>
</Rows>
</SubOrderBlock>
</SubOrderBlocks>
</BasePrintBlock>
</StandardPrintBlock>
</InvoicePri

--- Match 3 at position 15290 ---
No="3">
<Text>480,00</Text>
</Column>
</Columns>
</Row>
<Row>
<Columns>
<Column ColumnNo="2">
<Text>Att betala (SEK)</Text>
</Column>
<Column ColumnNo="3">
<Text>600,00</Text>
</Column>
</Columns>
</Row>
</Rows>
</SubOrderBlock>
</Su

In [19]:
# Let's extract the invoice data properly by targeting the right XML elements
import xml.etree.ElementTree as ET
import pandas as pd
import tempfile

# Read and clean the XML
with open('/Users/jiechen/Apoex/Email/Email_Execution_Tree/fastAPI/data/test_data/1.xml', 'r') as f:
    content = f.read()

xml_start = content.find('<CUSIN')
clean_xml = content[xml_start:]

# Parse the XML properly
with tempfile.NamedTemporaryFile(mode='w', suffix='.xml', delete=False) as temp_file:
    temp_file.write(clean_xml)
    temp_path = temp_file.name

tree = ET.parse(temp_path)
root = tree.getroot()

# Extract invoice data from each customer
invoice_data = []

for cusin_info in root.findall('CUSINInfo'):
    customer_no = cusin_info.find('CustomerNo')
    customer_name = cusin_info.find('CustomerInfo/Name')
    
    customer_no_text = customer_no.text if customer_no is not None else None
    customer_name_text = customer_name.text if customer_name is not None else None
    
    # Look for invoice data in the print blocks
    for invoice in cusin_info.findall('.//InvoiceWithDistribution'):
        invoice_no = invoice.find('InvoiceNo')
        invoice_date = invoice.find('InvoiceDate')
        due_date = invoice.find('PreferredDueDate')
        
        # Look for "Att betala (SEK)" amount in the print structure
        att_betala_amount = None
        
        # Search through all Row elements for the "Att betala (SEK)" text
        for row in invoice.findall('.//Row'):
            columns = row.findall('Columns/Column')
            for i, col in enumerate(columns):
                text_elem = col.find('Text')
                if text_elem is not None and text_elem.text == 'Att betala (SEK)':
                    # The amount should be in the next column
                    if i + 1 < len(columns):
                        amount_col = columns[i + 1]
                        amount_text = amount_col.find('Text')
                        if amount_text is not None:
                            att_betala_amount = amount_text.text
                            break
            if att_betala_amount:
                break
        
        invoice_data.append({
            'CustomerNo': customer_no_text,
            'CustomerName': customer_name_text,
            'InvoiceNo': invoice_no.text if invoice_no is not None else None,
            'InvoiceDate': invoice_date.text if invoice_date is not None else None,
            'DueDate': due_date.text if due_date is not None else None,
            'Att betala (SEK)': att_betala_amount
        })

# Create DataFrame
df_invoices = pd.DataFrame(invoice_data)

print("🎉 Successfully extracted invoice data with 'Att betala (SEK)'!")
print(f"Found {len(df_invoices)} invoices")
print("\nInvoice data:")
print(df_invoices)

# Cleanup
import os
os.unlink(temp_path)

🎉 Successfully extracted invoice data with 'Att betala (SEK)'!
Found 6 invoices

Invoice data:
  CustomerNo        CustomerName InvoiceNo InvoiceDate     DueDate  \
0       6892   Camilla Vanhatalo       667  2023-07-14  2023-08-13   
1       8613    Daria Kowalewska       662  2023-07-14  2023-08-13   
2       8614  Alexandra Elveving       663  2023-07-14  2023-08-13   
3       8617       Malin Nilsson       664  2023-07-14  2023-08-13   
4       8619    Johanna Lindelöf       665  2023-07-14  2023-08-13   
5       9286     Alexander Moore       666  2023-07-14  2023-08-13   

  Att betala (SEK)  
0          3043,00  
1          3157,00  
2           600,00  
3           900,00  
4           600,00  
5          3329,00  


In [20]:
# Create a reusable function for parsing Payex XML invoices
def parse_payex_xml(file_path):
    """Parse Payex XML invoice file and extract invoice data including 'Att betala (SEK)'"""
    import xml.etree.ElementTree as ET
    import pandas as pd
    import tempfile
    import os
    
    # Read and clean the XML
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
    
    # Remove any header comments
    xml_start = content.find('<CUSIN')
    if xml_start == -1:
        xml_start = content.find('<?xml')
        if xml_start == -1:
            xml_start = content.find('<')
    
    if xml_start > 0:
        content = content[xml_start:]
    
    # Parse XML
    with tempfile.NamedTemporaryFile(mode='w', suffix='.xml', delete=False) as temp_file:
        temp_file.write(content)
        temp_path = temp_file.name
    
    try:
        tree = ET.parse(temp_path)
        root = tree.getroot()
        
        # Extract invoice data
        invoice_data = []
        
        for cusin_info in root.findall('CUSINInfo'):
            # Get customer info
            customer_no = cusin_info.find('CustomerNo')
            customer_info = cusin_info.find('CustomerInfo')
            
            customer_no_text = customer_no.text if customer_no is not None else None
            customer_name = None
            customer_email = None
            
            if customer_info is not None:
                name_elem = customer_info.find('Name')
                email_elem = customer_info.find('.//E-MailAddress')
                customer_name = name_elem.text if name_elem is not None else None
                customer_email = email_elem.text if email_elem is not None else None
            
            # Get invoice details
            for invoice in cusin_info.findall('.//InvoiceWithDistribution'):
                invoice_no = invoice.find('InvoiceNo')
                invoice_date = invoice.find('InvoiceDate')
                due_date = invoice.find('PreferredDueDate')
                
                # Extract "Att betala (SEK)" amount
                att_betala_amount = None
                for row in invoice.findall('.//Row'):
                    columns = row.findall('Columns/Column')
                    for i, col in enumerate(columns):
                        text_elem = col.find('Text')
                        if text_elem is not None and text_elem.text == 'Att betala (SEK)':
                            if i + 1 < len(columns):
                                amount_col = columns[i + 1]
                                amount_text = amount_col.find('Text')
                                if amount_text is not None:
                                    att_betala_amount = amount_text.text
                                    break
                    if att_betala_amount:
                        break
                
                invoice_data.append({
                    'CustomerNo': customer_no_text,
                    'CustomerName': customer_name,
                    'CustomerEmail': customer_email,
                    'InvoiceNo': invoice_no.text if invoice_no is not None else None,
                    'InvoiceDate': invoice_date.text if invoice_date is not None else None,
                    'DueDate': due_date.text if due_date is not None else None,
                    'Att betala (SEK)': att_betala_amount
                })
        
        return pd.DataFrame(invoice_data)
        
    finally:
        os.unlink(temp_path)

# Test the function
result_df = parse_payex_xml('/Users/jiechen/Apoex/Email/Email_Execution_Tree/fastAPI/data/test_data/1.xml')
print("📊 Complete Payex Invoice Data:")
print("=" * 50)
print(result_df)
print(f"\nTotal invoices: {len(result_df)}")
print(f"Total amount: {result_df['Att betala (SEK)'].str.replace(',', '.').astype(float).sum():.2f} SEK")

📊 Complete Payex Invoice Data:
  CustomerNo        CustomerName                    CustomerEmail InvoiceNo  \
0       6892   Camilla Vanhatalo             cvanhatalo@gmail.com       667   
1       8613    Daria Kowalewska          daria.kowalewska1@wp.pl       662   
2       8614  Alexandra Elveving             elveving@icloude.com       663   
3       8617       Malin Nilsson          gullemulle2@outlook.com       664   
4       8619    Johanna Lindelöf  johannacamaralindelof@gmail.com       665   
5       9286     Alexander Moore      alexandermoore776@gmail.com       666   

  InvoiceDate     DueDate Att betala (SEK)  
0  2023-07-14  2023-08-13          3043,00  
1  2023-07-14  2023-08-13          3157,00  
2  2023-07-14  2023-08-13           600,00  
3  2023-07-14  2023-08-13           900,00  
4  2023-07-14  2023-08-13           600,00  
5  2023-07-14  2023-08-13          3329,00  

Total invoices: 6
Total amount: 11629.00 SEK


In [ ]:
result_df['']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   CustomerNo        6 non-null      object
 1   CustomerName      6 non-null      object
 2   CustomerEmail     6 non-null      object
 3   InvoiceNo         6 non-null      object
 4   InvoiceDate       6 non-null      object
 5   DueDate           6 non-null      object
 6   Att betala (SEK)  6 non-null      object
dtypes: object(7)
memory usage: 468.0+ bytes


In [33]:
# Updated reusable function with camelCase naming and float conversion
def parse_payex_xml(file_path):
    """Parse Payex XML invoice file and extract invoice data with camelCase columns"""
    import xml.etree.ElementTree as ET
    import pandas as pd
    import tempfile
    import os
    
    # Read and clean the XML
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
    
    # Remove any header comments
    xml_start = content.find('<CUSIN')
    if xml_start == -1:
        xml_start = content.find('<?xml')
        if xml_start == -1:
            xml_start = content.find('<')
    
    if xml_start > 0:
        content = content[xml_start:]
    
    # Parse XML
    with tempfile.NamedTemporaryFile(mode='w', suffix='.xml', delete=False) as temp_file:
        temp_file.write(content)
        temp_path = temp_file.name
    
    try:
        tree = ET.parse(temp_path)
        root = tree.getroot()
        
        # Extract invoice data
        invoice_data = []
        
        for cusin_info in root.findall('CUSINInfo'):
            # Get customer info
            customer_no = cusin_info.find('CustomerNo')
            customer_info = cusin_info.find('CustomerInfo')
            
            customer_no_text = customer_no.text if customer_no is not None else None
            customer_name = None
            customer_email = None
            
            if customer_info is not None:
                name_elem = customer_info.find('Name')
                email_elem = customer_info.find('.//E-MailAddress')
                customer_name = name_elem.text if name_elem is not None else None
                customer_email = email_elem.text if email_elem is not None else None
            
            # Get invoice details
            for invoice in cusin_info.findall('.//InvoiceWithDistribution'):
                invoice_no = invoice.find('InvoiceNo')
                invoice_date = invoice.find('InvoiceDate')
                due_date = invoice.find('PreferredDueDate')
                
                # Extract "Att betala (SEK)" amount and convert to float
                att_betala_amount = None
                invoice_amount_float = None
                
                for row in invoice.findall('.//Row'):
                    columns = row.findall('Columns/Column')
                    for i, col in enumerate(columns):
                        text_elem = col.find('Text')
                        if text_elem is not None and text_elem.text == 'Att betala (SEK)':
                            if i + 1 < len(columns):
                                amount_col = columns[i + 1]
                                amount_text = amount_col.find('Text')
                                if amount_text is not None:
                                    att_betala_amount = amount_text.text
                                    # Convert to float (replace comma with dot for Swedish number format)
                                    try:
                                        invoice_amount_float = float(att_betala_amount.replace(',', '.'))
                                    except (ValueError, AttributeError):
                                        invoice_amount_float = None
                                    break
                    if att_betala_amount:
                        break
                
                invoice_data.append({
                    'customerNo': customer_no_text,
                    'customerName': customer_name,
                    'customerEmail': customer_email,
                    'invoiceNo': invoice_no.text if invoice_no is not None else None,
                    'invoiceDate': invoice_date.text if invoice_date is not None else None,
                    'dueDate': due_date.text if due_date is not None else None,
                    'invoiceAmount': invoice_amount_float  # Float value, camelCase name
                })
        
        return pd.DataFrame(invoice_data)
        
    finally:
        os.unlink(temp_path)

# Test the updated function
result_df = parse_payex_xml('/Users/jiechen/Apoex/Email/Email_Execution_Tree/fastAPI/data/test_data/1.xml')
print("📊 Updated Payex Invoice Data (camelCase + float):")
print("=" * 60)
print(result_df)
print(f"\nData types:")
for col in result_df.columns:
    print(f"  {col}: {result_df[col].dtype}")
print(f"\nTotal invoices: {len(result_df)}")
print(f"Total amount: {result_df['invoiceAmount'].sum():.2f} SEK")

📊 Updated Payex Invoice Data (camelCase + float):
  customerNo      customerName                    customerEmail invoiceNo  \
0      33158  Hallsta Ridklubb  inbox.lev.1436124@arkivplats.se      4596   
1      33158  Hallsta Ridklubb  inbox.lev.1436124@arkivplats.se      4597   
2      33158  Hallsta Ridklubb  inbox.lev.1436124@arkivplats.se      4598   

  invoiceDate     dueDate  invoiceAmount  
0  2025-09-26  2025-10-26         3924.0  
1  2025-09-26  2025-10-26         3843.0  
2  2025-09-26  2025-10-26         1591.0  

Data types:
  customerNo: object
  customerName: object
  customerEmail: object
  invoiceNo: object
  invoiceDate: object
  dueDate: object
  invoiceAmount: float64

Total invoices: 3
Total amount: 9358.00 SEK


In [ ]:
# Simplified function without customerEmail and dueDate
def parse_payex_xml(file_path):
    """Parse Payex XML invoice file and extract essential invoice data with camelCase columns"""
    import xml.etree.ElementTree as ET
    import pandas as pd
    import tempfile
    import os
    
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()

    xml_start = content.find('<CUSIN')
    if xml_start == -1:
        xml_start = content.find('<?xml')
        if xml_start == -1:
            xml_start = content.find('<')
    
    if xml_start > 0:
        content = content[xml_start:]
    
    with tempfile.NamedTemporaryFile(mode='w', suffix='.xml', delete=False) as temp_file:
        temp_file.write(content)
        temp_path = temp_file.name
    
    try:
        tree = ET.parse(temp_path)
        root = tree.getroot()
        
        invoice_data = []
        
        for cusin_info in root.findall('CUSINInfo'):
            customer_no = cusin_info.find('CustomerNo')
            customer_info = cusin_info.find('CustomerInfo')
            
            customer_no_text = customer_no.text if customer_no is not None else None
            customer_name = None
            
            if customer_info is not None:
                name_elem = customer_info.find('Name')
                customer_name = name_elem.text if name_elem is not None else None
            
            for invoice in cusin_info.findall('.//InvoiceWithDistribution'):
                invoice_no = invoice.find('InvoiceNo')
                invoice_date = invoice.find('InvoiceDate')
                
                invoice_amount_float = None
                
                for row in invoice.findall('.//Row'):
                    columns = row.findall('Columns/Column')
                    for i, col in enumerate(columns):
                        text_elem = col.find('Text')
                        if text_elem is not None and text_elem.text == 'Att betala (SEK)':
                            if i + 1 < len(columns):
                                amount_col = columns[i + 1]
                                amount_text = amount_col.find('Text')
                                if amount_text is not None:
                                    try:
                                        invoice_amount_float = float(amount_text.text.replace(',', '.'))
                                    except (ValueError, AttributeError):
                                        invoice_amount_float = None
                                    break
                    if invoice_amount_float is not None:
                        break
                
                invoice_data.append({
                    'customerNo': customer_no_text,
                    'customerName': customer_name,
                    'invoiceNo': invoice_no.text if invoice_no is not None else None,
                    'invoiceDate': invoice_date.text if invoice_date is not None else None,
                    'invoiceAmount': invoice_amount_float
                })
        
        return pd.DataFrame(invoice_data)
        
    finally:
        os.unlink(temp_path)

# Test the simplified function
result_df = parse_payex_xml('/Users/jiechen/Apoex/Email/Email_Execution_Tree/fastAPI/data/test_data/1.xml')
print("📊 Simplified Payex Invoice Data:")
print("=" * 40)
print(result_df)
print(f"\nColumns: {list(result_df.columns)}")
print(f"Total invoices: {len(result_df)}")
print(f"Total amount: {result_df['invoiceAmount'].sum():.2f} SEK")

📊 Simplified Payex Invoice Data:
  customerNo        customerName invoiceNo invoiceDate  invoiceAmount
0       6892   Camilla Vanhatalo       667  2023-07-14         3043.0
1       8613    Daria Kowalewska       662  2023-07-14         3157.0
2       8614  Alexandra Elveving       663  2023-07-14          600.0
3       8617       Malin Nilsson       664  2023-07-14          900.0
4       8619    Johanna Lindelöf       665  2023-07-14          600.0
5       9286     Alexander Moore       666  2023-07-14         3329.0

Columns: ['customerNo', 'customerName', 'invoiceNo', 'invoiceDate', 'invoiceAmount']
Total invoices: 6
Total amount: 11629.00 SEK


In [ ]:
# Enhanced function with customerId parameter for filtering


# Test with all customers
print("📊 All customers:")
all_customers = parse_payex_xml('/Users/jiechen/Apoex/Email/Email_Execution_Tree/fastAPI/data/test_data/1.xml')
print(all_customers)

print("\n" + "="*50)

# Test with specific customer
print("📊 Customer 8619 only:")
specific_customer = parse_payex_xml('/Users/jiechen/Apoex/Email/Email_Execution_Tree/fastAPI/data/test_data/1.xml', customer_id='8619')
print(specific_customer)

print("\n" + "="*50)

# Test with non-existent customer
print("📊 Non-existent customer 9999:")
no_customer = parse_payex_xml('/Users/jiechen/Apoex/Email/Email_Execution_Tree/fastAPI/data/test_data/1.xml', customer_id='9999')
print(no_customer)
print(f"Empty DataFrame: {len(no_customer) == 0}")

📊 All customers:
  customerNo        customerName invoiceNo invoiceDate  invoiceAmount
0       6892   Camilla Vanhatalo       667  2023-07-14         3043.0
1       8613    Daria Kowalewska       662  2023-07-14         3157.0
2       8614  Alexandra Elveving       663  2023-07-14          600.0
3       8617       Malin Nilsson       664  2023-07-14          900.0
4       8619    Johanna Lindelöf       665  2023-07-14          600.0
5       9286     Alexander Moore       666  2023-07-14         3329.0

📊 Customer 8613 only:
  customerNo      customerName invoiceNo invoiceDate  invoiceAmount
0       8613  Daria Kowalewska       662  2023-07-14         3157.0

📊 Non-existent customer 9999:
Empty DataFrame
Columns: []
Index: []
Empty DataFrame: True


In [34]:
# Simplified function with only customerNo and invoiceAmount
def parse_payex_xml(file_path, customer_id=None):
    """Parse Payex XML invoice file and extract customerNo and invoiceAmount only
    
    Args:
        file_path (str): Path to the XML file
        customer_id (str, optional): Specific customer ID to filter. If None, returns all customers.
    
    Returns:
        pd.DataFrame: Invoice data with columns: customerNo, invoiceAmount
    """
    import xml.etree.ElementTree as ET
    import pandas as pd
    import tempfile
    import os
    
    # Read and clean the XML
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
    
    # Remove any header comments
    xml_start = content.find('<CUSIN')
    if xml_start == -1:
        xml_start = content.find('<?xml')
        if xml_start == -1:
            xml_start = content.find('<')
    
    if xml_start > 0:
        content = content[xml_start:]
    
    # Parse XML
    with tempfile.NamedTemporaryFile(mode='w', suffix='.xml', delete=False) as temp_file:
        temp_file.write(content)
        temp_path = temp_file.name
    
    try:
        tree = ET.parse(temp_path)
        root = tree.getroot()
        
        # Extract invoice data
        invoice_data = []
        
        for cusin_info in root.findall('CUSINInfo'):
            # Get customer number
            customer_no = cusin_info.find('CustomerNo')
            customer_no_text = customer_no.text if customer_no is not None else None
            
            # Skip if customer_id is specified and doesn't match
            if customer_id is not None and customer_no_text != str(customer_id):
                continue
            
            # Get invoice details
            for invoice in cusin_info.findall('.//InvoiceWithDistribution'):
                invoice_no = invoice.find('InvoiceNo')
                invoice_amount_float = None
                
                for row in invoice.findall('.//Row'):
                    columns = row.findall('Columns/Column')
                    for i, col in enumerate(columns):
                        text_elem = col.find('Text')
                        if text_elem is not None and text_elem.text == 'Att betala (SEK)':
                            if i + 1 < len(columns):
                                amount_col = columns[i + 1]
                                amount_text = amount_col.find('Text')
                                if amount_text is not None:
                                    # Fixed type warning issue
                                    amount_str = amount_text.text
                                    if amount_str is not None:
                                        invoice_amount_float = float(amount_str.replace(',', '.'))
                                    break
                    if invoice_amount_float is not None:
                        break
                
                invoice_data.append({
                    'customerNo': customer_no_text,
                    'invoiceNo': invoice_no.text if invoice_no is not None else None,
                    'invoiceAmount': invoice_amount_float
                })
        
        return pd.DataFrame(invoice_data)
        
    finally:
        os.unlink(temp_path)

# Test the simplified function
print("📊 Simplified Payex Invoice Data (only customerNo and invoiceAmount):")
print("=" * 70)

# All customers
all_data = parse_payex_xml('/Users/jiechen/Apoex/Email/Email_Execution_Tree/fastAPI/data/test_data/1.xml')
print("All customers:")
print(all_data)
print(f"Total: {all_data['invoiceAmount'].sum():.2f} SEK")

print("\n" + "-"*40)

# Specific customer
customer_data = parse_payex_xml('/Users/jiechen/Apoex/Email/Email_Execution_Tree/fastAPI/data/test_data/1.xml', customer_id='8613')
print("Customer 8613 only:")
print(customer_data)

📊 Simplified Payex Invoice Data (only customerNo and invoiceAmount):
All customers:
  customerNo invoiceNo  invoiceAmount
0      33158      4596         3924.0
1      33158      4597         3843.0
2      33158      4598         1591.0
Total: 9358.00 SEK

----------------------------------------
Customer 8613 only:
Empty DataFrame
Columns: []
Index: []


In [30]:
customer_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   customerNo     1 non-null      object 
 1   invoiceAmount  1 non-null      float64
dtypes: float64(1), object(1)
memory usage: 148.0+ bytes


In [ ]:
def parse_payex_xml(file_path, customer_id=None):
    """Parse Payex XML invoice file and extract customerNo and invoiceAmount with proper types
    
    Args:
        file_path (str): Path to the XML file
        customer_id (str, optional): Specific customer ID to filter. If None, returns all customers.
    
    Returns:
        pd.DataFrame: Invoice data with columns: customerNo (int), invoiceAmount (float)
    """
    import xml.etree.ElementTree as ET
    import pandas as pd
    import tempfile
    import os
    
    # Read and clean the XML
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
    
    # Remove any header comments
    xml_start = content.find('<CUSIN')
    if xml_start == -1:
        xml_start = content.find('<?xml')
        if xml_start == -1:
            xml_start = content.find('<')
    
    if xml_start > 0:
        content = content[xml_start:]
    
    # Parse XML
    with tempfile.NamedTemporaryFile(mode='w', suffix='.xml', delete=False) as temp_file:
        temp_file.write(content)
        temp_path = temp_file.name
    
    try:
        tree = ET.parse(temp_path)
        root = tree.getroot()
        
        # Extract invoice data
        invoice_data = []
        
        for cusin_info in root.findall('CUSINInfo'):
            # Get customer number and convert to int
            customer_no = cusin_info.find('CustomerNo')
            customer_no_int = None
            if customer_no is not None and customer_no.text is not None:
                try:
                    customer_no_int = int(customer_no.text)
                except ValueError:
                    customer_no_int = None
            
            # Skip if customer_id is specified and doesn't match
            if customer_id is not None:
                try:
                    if customer_no_int != int(customer_id):
                        continue
                except (ValueError, TypeError):
                    continue
            
            # Get invoice details
            for invoice in cusin_info.findall('.//InvoiceWithDistribution'):
                # Extract "Att betala (SEK)" amount and convert to float
                invoice_amount_float = None
                
                for row in invoice.findall('.//Row'):
                    columns = row.findall('Columns/Column')
                    for i, col in enumerate(columns):
                        text_elem = col.find('Text')
                        if text_elem is not None and text_elem.text == 'Att betala (SEK)':
                            if i + 1 < len(columns):
                                amount_col = columns[i + 1]
                                amount_text = amount_col.find('Text')
                                if amount_text is not None:
                                    amount_str = amount_text.text
                                    if amount_str is not None:
                                        try:
                                            invoice_amount_float = float(amount_str.replace(',', '.'))
                                        except ValueError:
                                            invoice_amount_float = None
                                    break
                    if invoice_amount_float is not None:
                        break
                
                # Only add if we have valid data
                if customer_no_int is not None and invoice_amount_float is not None:
                    invoice_data.append({
                        'customerNo': customer_no_int,
                        'invoiceAmount': invoice_amount_float
                    })
        
        return pd.DataFrame(invoice_data)
        
    finally:
        os.unlink(temp_path)
        


all_data = parse_payex_xml('/Users/jiechen/Apoex/Email/Email_Execution_Tree/fastAPI/data/test_data/1.xml')

print(all_data)


📊 Payex Invoice Data with Proper Types:
All customers:
   customerNo  invoiceAmount
0        6892         3043.0
1        8613         3157.0
2        8614          600.0
3        8617          900.0
4        8619          600.0
5        9286         3329.0

Data types:
  customerNo: int64
  invoiceAmount: float64
Total: 11629.00 SEK

----------------------------------------
Customer 8613 only (using int customer_id):
   customerNo  invoiceAmount
0        8613         3157.0
Data types:
  customerNo: int64
  invoiceAmount: float64
